# Assignment 3 - 2 (MLFFNN)

Team members:
- N Sowmya Manojna (BE17B007)
- Thakkar Riya Anandbhai (PH17B010)
- Chaithanya Krishna Moorthy (PH17B011)

## Import Essential Libraries

In [1]:
import wandb
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 18
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = 12,8
plt.rcParams['font.serif'] = "Cambria"
plt.rcParams['font.family'] = "serif"

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import warnings
warnings.filterwarnings("ignore")

## Reading the data, Splitting it

In [2]:
# Get the data
df = pd.read_csv("../datasets/2A/train_new.csv")
df_test = pd.read_csv("../datasets/2A/dev_new.csv")
display(df.head())

# Split dev into test and validation
df_val, df_test = train_test_split(df_test, test_size=0.3, random_state=42)
display(df_val.head())
display(df_test.head())

,class,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,0,0.301071,0.207291,0.163574,0.080124,0.054138,0.041107,0.048340,0.104355,0.028275,...,0.211029,0.176437,0.016968,0.029282,0.108093,0.173035,0.205368,0.192215,0.143951,0.131088
1,0,0.149094,0.178650,0.383118,0.217911,0.069717,0.001373,0.000122,0.000015,0.127670,...,0.003387,0.000168,0.121246,0.041306,0.045090,0.160141,0.290756,0.318909,0.022049,0.000504
2,0,0.033218,0.092514,0.107178,0.058289,0.397430,0.066330,0.238052,0.006989,0.000793,...,0.231079,0.003769,0.001465,0.004150,0.006897,0.028931,0.283752,0.127716,0.543488,0.003601
3,0,0.009476,0.132507,0.375351,0.090302,0.271347,0.112000,0.008942,0.000076,0.007553,...,0.096375,0.010620,0.007843,0.000443,0.030304,0.102646,0.246140,0.513321,0.088379,0.010925
4,0,0.169357,0.262299,0.253281,0.152527,0.092209,0.038757,0.016174,0.015396,0.082397,...,0.020676,0.015976,0.072647,0.077484,0.163315,0.186646,0.181931,0.196167,0.090378,0.031433


,class,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
266,3,0.207474,0.264450,0.094421,0.057404,0.023254,0.015213,0.019867,0.317917,0.126999,...,0.015518,0.348007,0.261932,0.197449,0.042343,0.037567,0.049713,0.029388,0.015289,0.366318
163,2,0.456207,0.072144,0.071976,0.078415,0.068756,0.058701,0.050797,0.143005,0.409531,...,0.063812,0.129852,0.376389,0.070969,0.031128,0.033157,0.049011,0.124374,0.134033,0.180939
152,2,0.696671,0.075043,0.042969,0.032593,0.024460,0.023254,0.031128,0.073883,0.671646,...,0.030426,0.074997,0.655106,0.072983,0.058853,0.044220,0.029526,0.026093,0.034698,0.078522
19,0,0.016556,0.238937,0.138321,0.069473,0.048096,0.048523,0.096878,0.343216,0.206787,...,0.040695,0.025192,0.937653,0.047638,0.010376,0.002518,0.000748,0.000732,0.000336,0.000000
223,3,0.339432,0.325745,0.071228,0.062363,0.028564,0.019608,0.009705,0.143356,0.230927,...,0.027664,0.162735,0.462479,0.205551,0.033432,0.008896,0.013138,0.028839,0.045441,0.202225


,class,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
219,3,0.158585,0.134903,0.102386,0.100128,0.078690,0.071960,0.087524,0.265823,0.155670,...,0.079834,0.239807,0.272598,0.156586,0.097137,0.072281,0.082184,0.072418,0.020569,0.226227
42,0,0.276459,0.036407,0.025345,0.107635,0.167801,0.138336,0.171936,0.076080,0.211014,...,0.167389,0.142151,0.187576,0.102997,0.024231,0.024338,0.070267,0.221176,0.156128,0.213287
211,3,0.027817,0.083313,0.217896,0.301102,0.243546,0.121887,0.003296,0.001144,0.001831,...,0.032196,0.001541,0.066345,0.129166,0.154922,0.081436,0.049789,0.040573,0.117401,0.360367
193,2,0.116089,0.124466,0.351288,0.126297,0.199615,0.081238,0.000748,0.000259,0.104462,...,0.133316,0.000885,0.094650,0.044830,0.315262,0.238190,0.034134,0.007690,0.033722,0.231522
56,0,0.128036,0.056778,0.075409,0.073059,0.085754,0.132904,0.143372,0.304688,0.147095,...,0.156586,0.266907,0.145401,0.037704,0.066437,0.096207,0.109009,0.115204,0.139206,0.290833


In [3]:
X_train = df.drop("class", axis=1)
y_train = df["class"].to_numpy().astype("int")

X_val = df_val.drop("class", axis=1)
y_val = df_val["class"].to_numpy().astype("int")

X_test = df_test.drop("class", axis=1)
y_test = df_test["class"].to_numpy().astype("int")

## Training the Model

In [4]:
# # parameters = {"hidden_layer_sizes":[(10,10),(15,5),(15,15),(25,25),(50,50),(100,100),(200,200)], \
# #               "activation":["logistic", "tanh", "relu"], "batch_size":[100, "auto"], \
# #               "alpha":[0.01, 0.0001], "learning_rate":["constant", "adaptive", "invscaling"], \
# #              }
# # parameters = {"hidden_layer_sizes":[(10,10),(25,25),(50,50),(100,100),(200,200),(256,256)], \
# #               "activation":["logistic", "tanh", "relu"], "batch_size":[50, 100, "auto"], \
# #               "alpha":[0.01, 0.0001], "learning_rate":["constant", "adaptive", "invscaling"], \
# #               "solver":["lbfgs", "sgd", "adam"]
# #              }


# parameters = {
#               "mlp__hidden_layer_sizes":[(25,25), (50,50), (75,75)], \
#               "mlp__batch_size":[50, 100, "auto"], "mlp__alpha":[0.01, 0.001], \
#               "mlp__learning_rate":["constant", "adaptive", "invscaling"], \
#               "mlp__early_stopping":[True, False]
#              }

# model = Pipeline([('scaler', StandardScaler()), ('mlp', MLPClassifier(max_iter=500, random_state=1))])

# clf = GridSearchCV(model, parameters, verbose=1, n_jobs=8)
# clf.fit(X_train, y_train)
# result_df = pd.DataFrame(clf.cv_results_)
# result_df.to_csv("../parameter_search/2A_MLFFNN_norm.csv")
# result_df = result_df.sort_values(by=["rank_test_score", "mean_fit_time"])
# display(result_df.head(10))

# best_mlp = clf.best_estimator_
# print("Best Parameters Choosen:")
# for i in clf.best_params_:
#     print("  - ", i, ": ", clf.best_params_[i], sep="")

# y_pred = best_mlp.predict(X_train)
# print("Accuracy:", 100*np.sum(y_pred==y_train)/y_train.size)
# conf_mat = confusion_matrix(y_train, y_pred)
# plt.figure()
# sns.heatmap(conf_mat, annot=True)
# plt.title("2A - Train Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_train_confmat.png")
# plt.show()

# y_val_pred = best_mlp.predict(X_val)
# print("Validation Accuracy:", 100*np.sum(y_val_pred==y_val)/y_val.size)
# val_conf_mat = confusion_matrix(y_val, y_val_pred)
# plt.figure()
# sns.heatmap(val_conf_mat, annot=True)
# plt.title("2A - Validation Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_val_confmat.png")
# plt.show()

# y_test_pred = best_mlp.predict(X_test)
# print("Validation Accuracy:", 100*np.sum(y_test_pred==y_test)/y_test.size)
# test_conf_mat = confusion_matrix(y_test, y_test_pred)
# plt.figure()
# sns.heatmap(test_conf_mat, annot=True)
# plt.title("2A - Test Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_test_confmat.png")
# plt.show()

In [5]:
# # parameters = {"hidden_layer_sizes":[(10,10),(15,5),(15,15),(25,25),(50,50),(100,100),(200,200)], \
# #               "activation":["logistic", "tanh", "relu"], "batch_size":[100, "auto"], \
# #               "alpha":[0.01, 0.0001], "learning_rate":["constant", "adaptive", "invscaling"], \
# #              }
# # parameters = {"hidden_layer_sizes":[(10,10),(25,25),(50,50),(100,100),(200,200),(256,256)], \
# #               "activation":["logistic", "tanh", "relu"], "batch_size":[50, 100, "auto"], \
# #               "alpha":[0.01, 0.0001], "learning_rate":["constant", "adaptive", "invscaling"], \
# #               "solver":["lbfgs", "sgd", "adam"]
# #              }


# parameters = {
#               "hidden_layer_sizes":[(25,25), (50,50), (75,75)], \
#               "batch_size":[50, 100, "auto"], "alpha":[0.01, 0.001], \
#               "learning_rate":["constant", "adaptive", "invscaling"], \
#               "early_stopping":[True, False]
#              }

# model = MLPClassifier(max_iter=500, random_state=1)

# clf = GridSearchCV(model, parameters, verbose=1, n_jobs=8)
# clf.fit(X_train, y_train)
# result_df = pd.DataFrame(clf.cv_results_)
# result_df.to_csv("../parameter_search/2A_MLFFNN.csv")
# result_df = result_df.sort_values(by=["rank_test_score", "mean_fit_time"])
# result_df.head()

# best_mlp = clf.best_estimator_
# print("Best Parameters Choosen:")
# for i in clf.best_params_:
#     print("  - ", i, ": ", clf.best_params_[i], sep="")

# y_pred = best_mlp.predict(X_train)
# print("Accuracy:", 100*np.sum(y_pred==y_train)/y_train.size)
# conf_mat = confusion_matrix(y_train, y_pred)
# plt.figure()
# sns.heatmap(conf_mat, annot=True)
# plt.title("2A - Train Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_train_confmat.png")
# plt.show()

# y_val_pred = best_mlp.predict(X_val)
# print("Validation Accuracy:", 100*np.sum(y_val_pred==y_val)/y_val.size)
# val_conf_mat = confusion_matrix(y_val, y_val_pred)
# plt.figure()
# sns.heatmap(val_conf_mat, annot=True)
# plt.title("2A - Validation Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_val_confmat.png")
# plt.show()

# y_test_pred = best_mlp.predict(X_test)
# print("Validation Accuracy:", 100*np.sum(y_test_pred==y_test)/y_test.size)
# test_conf_mat = confusion_matrix(y_test, y_test_pred)
# plt.figure()
# sns.heatmap(test_conf_mat, annot=True)
# plt.title("2A - Test Confusion Matrix (MLFFNN)")
# plt.xlabel("Predicted Class")
# plt.ylabel("Actual Class")
# plt.savefig("images/2A_MLFFNN_test_confmat.png")
# plt.show()

## Wandb

In [6]:
sweep_config = {'name': 'test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'val_accuracy', 'goal': 'maximize'}
parameters_dict = {
                   "hidden_layer_sizes": {"values": [(25,25), (50,50), (75,75)]},
                   "batch_size": {"values": [50, 100, "auto"]},
                   "alpha": {"values": [0.01, 0.001]},
                   "learning_rate": {"values": ["constant", "adaptive", "invscaling"]},
                   "early_stopping":{"values":[True, False]}
                  }
sweep_config['parameters'] = parameters_dict

In [7]:
def mlp_2a_sweep(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'hls_{}_bs_{}_a_{}_lr_{}_es_{}'.format(config.hidden_layer_sizes, config.batch_size, \
                                                                config.alpha, config.learning_rate, \
                                                                config.early_stopping)
        

        mlp = MLPClassifier(random_state=1, activation="relu", alpha=config.alpha, \
                            batch_size=config.batch_size, early_stopping=config.early_stopping, \
                            max_iter=500, hidden_layer_sizes=config.hidden_layer_sizes, \
                            learning_rate=config.learning_rate)

        mlp.fit(X_train, y_train)
        
        y_pred = mlp.predict(X_train)
        acc = 100*np.sum(y_pred==y_train)/y_train.size
        y_val_pred = mlp.predict(X_val)
        val_acc = 100*np.sum(y_val_pred==y_val)/y_val.size
        y_test_pred = mlp.predict(X_test)
        test_acc = 100*np.sum(y_test_pred==y_test)/y_test.size
        
        wandb.log({"accuracy":acc, "val_accuracy":val_acc, "test_accuracy":test_acc})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = 'PRML-A3')
wandb.agent(sweep_id, function=mlp_2a_sweep)

Create sweep with ID: gs0l7xqc
Sweep URL: https://wandb.ai/sowmyamanojna/PRML-A3/sweeps/gs0l7xqc


wandb: Agent Starting Run: 21a97ugv with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: 50
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [25, 25]
wandb: 	learning_rate: constant
wandb: Currently logged in as: sowmyamanojna (use `wandb login --relogin` to force relogin)


wandb: Agent Starting Run: bq0nt78i with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: 50
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [25, 25]
wandb: 	learning_rate: adaptive


wandb: Agent Starting Run: khazkoom with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: 50
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [25, 25]
wandb: 	learning_rate: invscaling


wandb: Agent Starting Run: cp8mcl4k with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: 50
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [50, 50]
wandb: 	learning_rate: constant


...

wandb: Agent Starting Run: 3m88ad29 with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: 100
wandb: 	early_stopping: False
wandb: 	hidden_layer_sizes: [75, 75]
wandb: 	learning_rate: invscaling


wandb: Agent Starting Run: 503ljh81 with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: auto
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [25, 25]
wandb: 	learning_rate: constant
wandb: ERROR Run 503ljh81 errored: UsageError("Error communicating with wandb process\ntry: wandb.init(settings=wandb.Settings(start_method='fork'))\nor:  wandb.init(settings=wandb.Settings(start_method='thread'))\nFor more info see: https://docs.wandb.ai/library/init#init-start-error")


Problem at: <ipython-input-7-f10fc451a855> 2 mlp_2a_sweep


wandb: Agent Starting Run: ywv5d70o with config:
wandb: 	alpha: 0.01
wandb: 	batch_size: auto
wandb: 	early_stopping: True
wandb: 	hidden_layer_sizes: [25, 25]
wandb: 	learning_rate: adaptive


wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.01725675336907 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.1790502007619743 seconds), retrying request
